Gares et stations du réseau ferré d'Île-de-France (par ligne)<br>
https://data.iledefrance-mobilites.fr/explore/dataset/emplacement-des-gares-idf/information/?location=16,48.85089,2.51991&basemap=jawg.streets

Stations de métro (Toulouse)<br>
https://data.toulouse-metropole.fr/explore/dataset/stations-de-metro/map/?location=16,43.60215,1.44379&basemap=jawg.streets

Itinéraires de métro dans OpenStreetMap<br>
https://www.data.gouv.fr/fr/datasets/itineraires-de-metro-dans-openstreetmap/

Itinéraires de tramways dans OpenStreetMap<br>
https://www.data.gouv.fr/fr/datasets/itineraires-de-tramways-dans-openstreetmap/

Using GeoPandas to convert Linestring UTM data to LatLong<br>
https://gis.stackexchange.com/questions/351846/using-geopandas-to-convert-linestring-utm-data-to-latlong

Converting UTM to Lat/Long with a geoJSON file in pyproj<br>
https://gis.stackexchange.com/questions/217012/converting-utm-to-lat-long-with-a-geojson-file-in-pyproj

Converting projected coordinates to lat/lon using Python?<br>
https://gis.stackexchange.com/questions/78838/converting-projected-coordinates-to-lat-lon-using-python

pyproj4 / pyproj<br>
https://github.com/pyproj4/pyproj

Turbo87 / utm<br>
https://github.com/Turbo87/utm

Universal Transverse Mercator coordinate system<br>
https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#From_UTM_coordinates_(E,_N,_Zone,_Hemi)_to_latitude,_longitude_(%CF%86,_%CE%BB)



In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import datetime
import math

import geopandas as gpd
import shapely.wkt
from shapely.geometry import Point, LineString
from shapely.geometry.polygon import Polygon
from branca.element import Template, MacroElement
import branca.colormap as cm
import folium
from folium import plugins

import json
import pyproj


## Importing data

<https://pyproj4.github.io/pyproj/v2.6.1rel/api/proj.html?highlight=projparams><br>
Performs cartographic transformations.<br>
Converts from longitude, latitude to native map projection x,y coordinates and vice versa using `PROJ`.

In [2]:
# read file
with open('data/transport/metro_lignes.json', 'r') as myfile:
    data = myfile.read()

# parse file
subway_lines_df = json.loads(data)

In [3]:
wgs84 = pyproj.Proj(projparams = 'epsg:4326') # http://epsg.io/4326
InputGrid = pyproj.Proj(projparams = 'epsg:3857') # https://epsg.io/3857

`epsg:4326`: WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS.<br>
The World Geodetic System (WGS) is a standard for use in cartography, geodesy, and satellite navigation including GPS. This standard includes the definition of the coordinate system's fundamental and derived constants, the ellipsoidal (normal) Earth Gravitational Model (EGM), a description of the associated World Magnetic Model (WMM), and a current list of local datum transformations.<br>
The latest revision is WGS 84 (also known as WGS 1984, EPSG:4326), established and maintained by the United States National Geospatial-Intelligence Agency since 1984, and last revised in 2004. Earlier schemes included WGS 72, WGS 66, and WGS 60. WGS 84 is the reference coordinate system used by the Global Positioning System.<br>
https://en.wikipedia.org/wiki/World_Geodetic_System


`epsg:3857`: WGS 84 / Pseudo-Mercator -- Spherical Mercator, Google Maps, OpenStreetMap, Bing, ArcGIS, ESRI.<br>
Web Mercator, Google Web Mercator, Spherical Mercator, WGS 84 Web Mercator or WGS 84/Pseudo-Mercator is a variant of the Mercator projection and is the de facto standard for Web mapping applications. It rose to prominence when Google Maps adopted it in 2005. It is used by virtually all major online map providers, including Google Maps, Mapbox, Bing Maps, OpenStreetMap, Mapquest, Esri, and many others. Its official EPSG identifier is EPSG:3857, although others have been used historically.<br>
https://en.wikipedia.org/wiki/Web_Mercator_projection

In [4]:
x1, y1 = -11705274.6374,4826473.6922
pyproj.transform(InputGrid, wgs84, x1, y1)

/Users/jingwen/github/dvf/_venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


(39.72785727727918, -105.15027111593008)

In [5]:
transformer = pyproj.Transformer.from_crs("epsg:3857", "epsg:4326")
transformer.transform(x1, y1)

(39.72785727727918, -105.15027111593008)

In [ ]:
# approach 1
i = 0
for feature in subway_lines_df['features']:
    if feature['geometry']['type'] == 'LineString':
        # all coordinates
        coords = feature['geometry']['coordinates']
        print('----- %d -----' % i)
        print(coords)
        
        # coordList is for each individual polygon
        for coordList in coords:
            lat_grid, lng_grid = coordList
            # do transformation
            coordList[0],coordList[1] = pyproj.transform(InputGrid, wgs84, lat_grid, lng_grid)
    elif feature['geometry']['type'] == 'MultiLineString':
        len_coord = len(feature['geometry']['coordinates'])
        for c in range(len_coord):
            coords = feature['geometry']['coordinates'][c]
            for coordList in coords:
                lat_grid, lng_grid = coordList
                coordList[0],coordList[1] = pyproj.transform(InputGrid, wgs84, lat_grid, lng_grid)
    i += 1



In [ ]:
subway_lines_df

In [6]:
# approach 2
i = 0
transformer = pyproj.Transformer.from_crs("epsg:3857", "epsg:4326")

for feature in subway_lines_df['features']:
    if feature['geometry']['type'] == 'LineString':
        # all coordinates
        coords = feature['geometry']['coordinates']
        print('----- %d -----' % i)
        print(coords)
        
        # coordList is for each individual polygon
        for coordList in coords:
            lat_grid, lng_grid = coordList
            # do transformation
            coordList[0],coordList[1] = transformer.transform(lat_grid, lng_grid)
    elif feature['geometry']['type'] == 'MultiLineString':
        len_coord = len(feature['geometry']['coordinates'])
        for c in range(len_coord):
            coords = feature['geometry']['coordinates'][c]
            for coordList in coords:
                lat_grid, lng_grid = coordList
                coordList[0],coordList[1] = transformer.transform(lat_grid, lng_grid)
    i += 1



----- 0 -----
[[263867.5465, 6229019.5306], [263818.6995, 6229006.8059], [263775.2849, 6228994.8069], [263726.2264, 6228980.884], [263666.2475, 6228964.3959], [263564.2231, 6228936.0102], [263453.1263, 6228904.8232], [263404.2125, 6228891.5586], [263380.2343, 6228888.2678], [263361.8888, 6228887.5083], [263344.4339, 6228888.6053], [263321.0791, 6228892.5205], [263300.4627, 6228898.444], [263284.455, 6228904.7557], [263269.4825, 6228912.5862], [263257.3153, 6228920.7036], [263240.8289, 6228933.2088], [263225.99, 6228947.2835], [263213.6892, 6228961.5101], [263201.6667, 6228978.8926], [263178.1004, 6229016.0541], [263162.9164, 6229040.3054], [263151.3726, 6229058.11], [263140.0402, 6229073.4337], [263128.4964, 6229087.3737], [263116.5407, 6229099.3897], [263107.6797, 6229107.3723], [263091.5383, 6229120.1478], [263077.1002, 6229130.0374], [263062.8847, 6229138.0706], [263046.7434, 6229145.6313], [263032.0381, 6229151.0656], [263010.8428, 6229157.1749], [262992.3193, 6229161.3097], [26296

----- 9 -----
[[286284.6648, 6275635.5902], [286281.3141, 6275667.6626], [286278.1971, 6275701.6017], [286274.2008, 6275738.8161], [286264.0707, 6275791.7107], [286260.4194, 6275826.957], [286233.2352, 6276089.4676], [286216.7154, 6276181.8549], [286191.0117, 6276292.2151], [286183.3418, 6276344.8417], [286167.1671, 6276523.2076], [286164.1948, 6276569.5392], [286158.1836, 6276650.3571], [286157.7494, 6276656.2462], [286155.3895, 6276681.9411], [286151.0591, 6276702.6634], [286141.1183, 6276724.3192], [286129.3296, 6276740.9175], [286113.3552, 6276757.3461], [286089.4661, 6276775.8453], [286062.5935, 6276790.7805], [286034.1069, 6276799.7416], [286006.7557, 6276803.119], [285919.8374, 6276794.7519], [285875.7438, 6276789.6264], [285841.8247, 6276786.6393], [285808.6849, 6276780.6823], [285779.419, 6276764.8476], [285758.5244, 6276743.65], [285720.3084, 6276663.0518], [285682.905, 6276637.4588], [285625.9095, 6276623.1179], [285375.0065, 6276596.9989], [285234.8775, 6276586.4257], [2851

----- 59 -----
[[535186.3633, 5745807.0375], [534838.7904, 5745253.9411], [534726.0461, 5745084.7307], [534702.7914, 5745029.5677], [534687.7633, 5744969.9039], [534682.6426, 5744906.0586], [534695.6002, 5744846.1402], [534712.7545, 5744805.998], [534736.2875, 5744770.8518], [534779.3124, 5744721.1972], [534827.4359, 5744672.5804], [534866.8541, 5744631.9761], [534897.0105, 5744596.7348], [534914.8662, 5744567.4948], [534933.8462, 5744516.357], [534940.1691, 5744479.1211], [534944.9781, 5744445.1094], [534943.4419, 5744354.2152], [534942.9855, 5744326.971], [534942.5625, 5744301.6422], [534942.0838, 5744273.2651], [534941.6163, 5744245.6062], [534940.9929, 5744208.2916], [534935.9501, 5744118.916], [534938.4548, 5744057.407], [534942.7183, 5743906.3334], [534946.737, 5743772.4664], [534948.2398, 5743694.857], [534956.3438, 5743443.3134], [534957.034, 5743403.8159], [534953.572, 5743363.6642], [534937.4306, 5743281.1433], [534932.6662, 5743228.449], [534934.1801, 5743159.4138], [534946.

In [7]:
subway_lines_df

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'france_subway_routes_line.9921604',
   'geometry': {'type': 'LineString',
    'coordinates': [[48.72909889953939, 2.370362500040685],
     [48.72902349968515, 2.3699236999738513],
     [48.728952399842406, 2.3695336999865124],
     [48.72886989984137, 2.3690929999828523],
     [48.728772199599014, 2.368554200356906],
     [48.72860399967895, 2.3676376995781743],
     [48.72841919996222, 2.3666397000436064],
     [48.72834059988068, 2.366200299902163],
     [48.728321100054025, 2.3659849000667066],
     [48.728316599591565, 2.365820099636258],
     [48.72832309993024, 2.36566329960173],
     [48.72834629967361, 2.3654534998637544],
     [48.72838139969408, 2.3652682995915195],
     [48.728418799987736, 2.3651244999757837],
     [48.728465199968014, 2.3649899997198687],
     [48.72851329994196, 2.3648806999026193],
     [48.728587399907866, 2.3647326000516187],
     [48.72867079986604, 2.364599299944923],
     [48.

In [ ]:
# # write reprojected json to new file
# with open('data/transport/metro_lignes_v2.json', 'w') as f:
#     f.write(json.dumps(subway_lines_df))